In [2]:
from datasets.get_datasets import *
from boostedrevision import *
from boostsrl import boostsrl
import random

## Dataset

In [29]:
target = 'function'

bk = [
    'location(+protein,+loc).',
    'location(+protein,-loc).',
    'location(-protein,+loc).',
    'interaction(+protein,+protein).',
    'interaction(+protein,-protein).',
    'interaction(-protein,+protein).',
    'proteinclass(+protein,+class).',
    'proteinclass(+protein,-class).',
    'proteinclass(-protein,+class).',
    'enzyme(+protein,+enz).',
    'enzyme(+protein,-enz).',
    'enzyme(-protein,+enz).',
    'function(+protein,+fun).',
    'function(+protein,-fun).',
    'function(-protein,+fun).',
    'complex(+protein,+com).',
    'complex(+protein,-com).',
    'complex(-protein,+com).',
    'phenotype(+protein,+phe).',
    'phenotype(+protein,-phe).',
    'phenotype(-protein,+phe).',]

[facts, pos, neg] = datasets.load('yeast', bk, target=target)

## Background  configuration

In [30]:
background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

## Learning

In [31]:
for i in range(len(pos)):
    [train_facts, test_facts] = datasets.get_kfold_small(i, facts)
    [train_pos, test_pos] = datasets.get_kfold_small(i, pos)
    [train_neg, test_neg] = datasets.get_kfold_small(i, neg)
    [model, learning_time, inference_time, t_results, structured, will] = learn_test_model(background, boostsrl, target, train_pos, train_neg, train_facts, test_pos, test_neg, test_facts, refine=None, trees=10, verbose=True)

WILL Produced-Tree #1
% FOR function(A, B):
%   if ( interaction(A, C) )
%   then if ( interaction(C, A) )
%   | then if ( interaction(C, C) )
%   | | then return 0.3581489350995082;  // std dev = 0,500, 382,000 (wgt'ed) examples reached here.  /* #neg=191 #pos=191 */
%   | | else if ( phenotype(A, D) )
%   | | | then return 0.35814893509951334;  // std dev = 0,500, 150,000 (wgt'ed) examples reached here.  /* #neg=75 #pos=75 */
%   | | | else return 0.35814893509951345;  // std dev = 0,500, 138,000 (wgt'ed) examples reached here.  /* #neg=69 #pos=69 */
%   | else if ( interaction(C, C) )
%   | | then return 0.3581489350995131;  // std dev = 0,500, 92,000 (wgt'ed) examples reached here.  /* #neg=46 #pos=46 */
%   | | else if ( interaction(C, E) )
%   | | | then if ( interaction(E, C) )
%   | | | | then return 0.3581489350995122;  // std dev = 0,500, 158,000 (wgt'ed) examples reached here.  /* #neg=79 #pos=79 */
%   | | | | else return 0.35814893509951334;  // std dev = 0,500, 116,000 (w

In [32]:
target = 'function'

bk = [
    'location(+protein,#loc).',
    'location(+protein,#loc).',
    'location(-protein,#loc).',
    'interaction(+protein,#protein).',
    'interaction(+protein,#protein).',
    'interaction(-protein,#protein).',
    'proteinclass(+protein,#class).',
    'proteinclass(+protein,#class).',
    'proteinclass(-protein,#class).',
    'enzyme(+protein,#enz).',
    'enzyme(+protein,#enz).',
    'enzyme(-protein,#enz).',
    'function(+protein,#fun).',
    'function(+protein,#fun).',
    'function(-protein,#fun).',
    'complex(+protein,#com).',
    'complex(+protein,#com).',
    'complex(-protein,#com).',
    'phenotype(+protein,#phe).',
    'phenotype(+protein,#phe).',
    'phenotype(-protein,#phe).',]

[facts, pos, neg] = datasets.load('yeast', bk, target=target)

background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

for i in range(len(pos)):
    [train_facts, test_facts] = datasets.get_kfold_small(i, facts)
    [train_pos, test_pos] = datasets.get_kfold_small(i, pos)
    [train_neg, test_neg] = datasets.get_kfold_small(i, neg)
    [model, learning_time, inference_time, t_results, structured, will] = learn_test_model(background, boostsrl, target, train_pos, train_neg, train_facts, test_pos, test_neg, test_facts, refine=None, trees=10, verbose=True)

WILL Produced-Tree #1
% FOR function(A, B):
%   if ( location(A, locationid725) )
%   then if ( location(A, locationid750005) )
%   | then if ( location(A, locationid755) )
%   | | then return 0.35814893509951223;  // std dev = 1,225, 6,000 (wgt'ed) examples reached here.  /* #neg=3 #pos=3 */
%   | | else if ( location(A, locationid750001) )
%   | | | then return 0.3581489350995123;  // std dev = 0,500, 8,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=4 */
%   | | | else if ( interaction(A, proteinygr158c) )
%   | | | | then return 0.3581489350995122;  // std dev = 1,000, 4,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=2 */
%   | | | | else if ( interaction(A, proteinydl111c) )
%   | | | | | then return 0.35814893509951223;  // std dev = 1,225, 6,000 (wgt'ed) examples reached here.  /* #neg=3 #pos=3 */
%   | | | | | else return 0.3581489350995122;  // std dev = 1,000, 4,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=2 */
%   | else if ( location(A, locationid755003) )
%

Exception: ('Encountered problems while running process: ', '(cd boostsrl; java -jar v1-0.jar -l -combine -train train/ -target function -trees 10 > train_output.txt 2>&1)')

In [144]:
def load(dataset, bk, target=None, seed=None):
    '''Load dataset from json and accept only predicates presented in bk'''
    pattern = '^(\w+)\(.*\).$'
    accepted = set()
    for line in bk:
        m = re.search(pattern, line)
        if m:
            relation = re.sub('[ _]', '', m.group(1))
            accepted.add(relation)
    data = datasets.get_json_dataset(dataset)
    facts = []
    pos = []
    neg = []
    for i in range(len(data[0])): #positives
        facts.append([])
        pos.append([])
        neg.append([])
        for relation, value in data[0][i].items():
            if relation in accepted or relation == 'accounttype':
                if relation == target or (target in ['fan', 'news', 'club'] and relation == 'accounttype'):
                    for example in value:
                        if relation == 'accounttype':
                            if example[1] == target:
                                pos[i].append(example[1] + '(' + example[0] + ').')
                            else:
                                facts[i].append(example[1] + '(' + example[0] + ').')
                        else:
                            pos[i].append(relation + '(' + ','.join(example)+ ').')
                else:
                    for example in value:
                        if relation == 'accounttype':
                            facts[i].append(example[1] + '(' + example[0] + ').')
                        else:
                            facts[i].append(relation + '(' + ','.join(example)+ ').')
    if target:
        for i in range(len(data[1])): #negatives
            if target in ['fan', 'news', 'club']:
                for j in data[0][i]['accounttype']:
                    if j[1] != target:
                        neg[i].append(target + '(' + j[0] + ').')
    return [facts, pos, neg]